In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from pycaret.anomaly import *

from pyod.models.abod import ABOD

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Random Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(69) # Seed 고정

## Data Load

In [ ]:
data_path = '/content/drive/MyDrive/AI_SPARK/dataset'
train = pd.read_csv(data_path+'/train_data.csv')
test = pd.read_csv(data_path+'/test_data.csv')

In [ ]:
train

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0
...,...,...,...,...,...,...,...,...
2458,2.28,50.20,0.7,29.53,2416.0,66.84,3.34,7
2459,2.04,46.94,0.7,26.34,2155.0,63.94,3.20,7
2460,1.19,35.74,0.7,15.39,1259.0,53.99,2.70,7
2461,1.21,36.00,0.7,15.64,1280.0,54.22,2.71,7


In [ ]:
test

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0
...,...,...,...,...,...,...,...,...
7384,2.12,48.08,0.7,27.45,2246.0,64.96,3.25,7
7385,1.48,39.63,0.7,19.19,1570.0,57.44,2.87,7
7386,1.56,40.61,0.7,20.15,1649.0,58.32,2.92,7
7387,1.59,40.99,0.7,20.52,1679.0,58.66,2.93,7


## Pre-Processing

#### 변수생성

In [ ]:
train['HP'] = ''
test['HP'] = ''

In [ ]:
# 마력 HP 변수 생성
train.loc[train['type'] == 0, 'HP'] = 30
train.loc[train['type'] == 1, 'HP'] = 20
train.loc[train['type'] == 2, 'HP'] = 10
train.loc[train['type'] == 3, 'HP'] = 50
train.loc[train['type'] == 4, 'HP'] = 30
train.loc[train['type'] == 5, 'HP'] = 30
train.loc[train['type'] == 6, 'HP'] = 30
train.loc[train['type'] == 7, 'HP'] = 30

test.loc[test['type'] == 0, 'HP'] = 30
test.loc[test['type'] == 1, 'HP'] = 20
test.loc[test['type'] == 2, 'HP'] = 10
test.loc[test['type'] == 3, 'HP'] = 50
test.loc[test['type'] == 4, 'HP'] = 30
test.loc[test['type'] == 5, 'HP'] = 30
test.loc[test['type'] == 6, 'HP'] = 30
test.loc[test['type'] == 7, 'HP'] = 30

In [ ]:
# 킬로와트 kw 변수 생성
train['kw'] = (1 / 0.746) * train['HP']
test['kw'] = (1 / 0.746) * test['HP']

In [ ]:
# 볼트 생성 -> V = W(와트) / A(전류)
train['volt'] = (0.001*train['kw']) / train['motor_current']
test['volt'] = (0.001*test['kw']) / test['motor_current']

In [ ]:
# 토크 생성 -> T = 716200 * HP(마력) / rpm(회전수)
train['torque'] = 716200 * train['HP'] / train['motor_rpm']
test['torque'] = 716200 * test['HP'] / test['motor_rpm']

In [ ]:
# 각속도 생성 -> 각속도 = kw / 토크T
train['각속도'] = train['kw'] / train['torque']
test['각속도'] = test['kw'] / test['torque']

In [ ]:
# 회전수 당 진동
train['회전수_진동'] = train['motor_rpm'] / train['motor_vibe']
test['회전수_진동'] = test['motor_rpm'] / test['motor_vibe']

In [ ]:
# 회전수 당 전류
train['회전수_전류'] = train['motor_rpm'] / train['motor_current']
test['회전수_전류'] = test['motor_rpm'] / test['motor_current']

In [ ]:
# Hz = 60*rpm
train['Hz'] = 60 * train['motor_rpm']
test['Hz'] = 60 * test['motor_rpm']

# 주기 생성 -> 주기 = 1 / Hz(주파수)
train['주기'] = 1 / train['Hz']
test['주기'] = 1 / test['Hz']

In [ ]:
train

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type,HP,kw,volt,torque,각속도,회전수_진동,회전수_전류,Hz,주기
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0,30,40.214477,0.001959,12789.285714,0.003144,573.378840,81.831466,100800.0,0.000010
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0,30,40.214477,0.001047,6838.319542,0.005881,837.866667,81.822917,188520.0,0.000005
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0,30,40.214477,0.001626,10620.860109,0.003786,648.397436,81.803478,121380.0,0.000008
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0,30,40.214477,0.001313,8573.822825,0.004690,739.233038,81.815214,150360.0,0.000007
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0,30,40.214477,0.001631,10652.454140,0.003775,646.474359,81.825558,121020.0,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458,2.28,50.20,0.7,29.53,2416.0,66.84,3.34,7,30,40.214477,0.001362,8893.211921,0.004522,723.353293,81.815103,144960.0,0.000007
2459,2.04,46.94,0.7,26.34,2155.0,63.94,3.20,7,30,40.214477,0.001527,9970.301624,0.004033,673.437500,81.814730,129300.0,0.000008
2460,1.19,35.74,0.7,15.39,1259.0,53.99,2.70,7,30,40.214477,0.002613,17065.925338,0.002356,466.296296,81.806368,75540.0,0.000013
2461,1.21,36.00,0.7,15.64,1280.0,54.22,2.71,7,30,40.214477,0.002571,16785.937500,0.002396,472.324723,81.841432,76800.0,0.000013


## Train

In [ ]:
angle = ABOD()
angle.fit(train)

ABOD(contamination=0.1, method='fast', n_neighbors=5)

## Predict

In [ ]:
pred_test = angle.predict(test)

In [ ]:
pred_test

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# 정상 (0), 이상 (1)
list(pred_test).count(0), list(pred_test).count(1)

(6732, 657)

## Submission

In [ ]:
submit = pd.read_csv(data_path+'/answer_sample.csv')

In [ ]:
submit['label'] = pred_test
submit.head()

,type,label
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [ ]:
submit.to_csv('/content/drive/MyDrive/AI_SPARK/submit/7.csv', index=False)